<a href="https://colab.research.google.com/github/MalikHasnat1999/Web-Scraping/blob/master/Disney_Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [114]:
# import libraries
import requests
from bs4 import BeautifulSoup

## Load the webpage

In [115]:
# request the webpage connection
r = requests.get("https://en.wikipedia.org/wiki/Tron:_Legacy")

# convert to beautiful soup
soup = BeautifulSoup(r.content)
contents = soup.prettify()
# print(contents)

### Task#1: Creating Dictionary of info box

In [116]:
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all("tr")


def get_content_value(td):
  if td.find("li"):
    return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in td.find_all("li")]
  else:
    return td.get_text(" ", strip=True).replace("\xa0", " ")


movie_info = {}
for index, tr in enumerate(info_rows):
  if index == 0:
    movie_info["title"] = tr.th.get_text(" ", strip=True)
  elif index == 1:
    continue
  else:
    content_key = tr.th.get_text(" ", strip=True)
    content_value = get_content_value(tr.find("td"))
    movie_info[content_key] = content_value

movie_info

{'title': 'Tron: Legacy',
 'Directed by': 'Joseph Kosinski',
 'Screenplay by': ['Edward Kitsis', 'Adam Horowitz'],
 'Story by': ['Edward Kitsis',
  'Adam Horowitz',
  'Brian Klugman',
  'Lee Sternthal'],
 'Based on': ['Steven Lisberger', 'Bonnie MacBird'],
 'Produced by': ['Sean Bailey', 'Jeffrey Silver', 'Steven Lisberger'],
 'Starring': ['Garrett Hedlund',
  'Jeff Bridges',
  'Olivia Wilde',
  'Bruce Boxleitner',
  'James Frain',
  'Beau Garrett',
  'Michael Sheen'],
 'Cinematography': 'Claudio Miranda',
 'Edited by': 'James Haygood',
 'Music by': 'Daft Punk',
 'Production companies': ['Walt Disney Pictures [1]',
  'Sean Bailey Productions [2]'],
 'Distributed by': 'Walt Disney Studios Motion Pictures [1]',
 'Release dates': ['November 30, 2010 ( 2010-11-30 ) (Tokyo)',
  'December 17, 2010 ( 2010-12-17 ) (United States) [3]'],
 'Running time': '125 minutes [4]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$170 million [5] [6]',
 'Box office': '$400.1 million [6]'

### Task#2: Get info box for all movies

In [117]:
# request the webpage connection
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# convert to beautiful soup
soup = BeautifulSoup(r.content)
contents = soup.prettify()
# print(contents)

In [118]:
movies = soup.select(".wikitable.sortable i a")
movies[8]['href']

'/wiki/The_Three_Caballeros'

In [119]:

def get_content_value(td):
  if td.find("li"):
    return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in td.find_all("li")]
  elif td.find('br'):
     return [text for text in td.stripped_strings]
  else:
    return td.get_text(" ", strip=True).replace("\xa0", " ")

def clean_tags(soup):
  """
    remove references [1], [2] etc
   """
  for tag in soup.find_all(['sup', 'span']):
    tag.decompose()

def get_info_box(url):

  r = requests.get(url)

  soup = BeautifulSoup(r.content)
  contents = soup.prettify()
  info_box = soup.find(class_="infobox vevent")
  info_rows = info_box.find_all("tr")

  clean_tags(soup)

  movie_info = {}
  for index, tr in enumerate(info_rows):
    if index == 0:
      movie_info["title"] = tr.th.get_text(" ", strip=True)
    # elif index == 1:
    #   continue
    else:
      header = tr.find('th')
      if header:
        content_key = tr.th.get_text(" ", strip=True)
        content_value = get_content_value(tr.find("td"))
        movie_info[content_key] = content_value

  return movie_info


In [120]:
get_info_box("https://en.wikipedia.org/wiki/One_Little_Indian_(film)")

{'title': 'One Little Indian',
 'Directed by': 'Bernard McEveety',
 'Written by': 'Harry Spalding',
 'Produced by': 'Winston Hibler',
 'Starring': ['James Garner',
  'Vera Miles',
  'Pat Hingle',
  "Clay O'Brien",
  'John Doucette',
  'Morgan Woodward',
  'Andrew Prine'],
 'Cinematography': 'Charles F. Wheeler',
 'Edited by': 'Robert Stafford',
 'Music by': 'Jerry Goldsmith',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['June 20, 1973'],
 'Running time': '90 Minutes',
 'Country': 'United States',
 'Language': 'English',
 'Box office': '$2 million'}

In [121]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = BeautifulSoup(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.wikipedia.org"
movie_info_list = []
for index, movie in enumerate(movies):
  if index % 10 == 0:
    print(index)
  try:
    relative_path = movie['href']
    full_path = base_path + relative_path
    title = movie['title']
  
    movie_info_list.append(get_info_box(full_path))
  except Exception as e:
    print(movie.get_text())
    print(e)
  

0
10
20
30
40
Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
The Beatles: Get Back – The Rooftop Concert
'NoneType' object has no attribute 'find'
490
500
Elemental
'NoneType' object has no attribute 'find_all'
61
'NoneType' object has no attribute 'find_all'
All Night Long
'NoneType' object has no attribute 'find'
Big Thunder Mountain Railroad
'NoneType' object has no attribute 'find_all'
510
Keeper of the Lost Cities
'NoneType' object has no attribute 'find_all'
Muppet Man
'NoneType' object has no attribute 'find_all'
520
Shrunk
'NoneType' object has no attribute 'find'
Sister Act 3
'NoneType' object has no attribute 'find'
The Graveyard Book
'NoneType' object has no attribute 'find_all'
The Thief
'NoneType' object has no attribute 'find_all'
5

In [122]:
len(movie_info_list)

520

### Task#3: Clean our data

* ~~Clean up references [1], [2] etc~~
* Convert running time into an integer
* Convert Dates into DateTime object
* ~~Split up long strings~~
* Convert Budget & Box office to number


#### Convert running time to an integer

In [123]:
[movie.get('Running time', 'N/A') for movie in movie_info_list]

['83 minutes',
 '88 minutes',
 '126 minutes',
 '74 minutes',
 '64 minutes',
 '70 minutes',
 '42 minutes',
 '70 min',
 '71 minutes',
 '75 minutes',
 '94 minutes',
 '73 minutes',
 '75 minutes',
 '82 minutes',
 '68 minutes',
 '74 minutes',
 '96 minutes',
 '75 minutes',
 '84 minutes',
 '77 minutes',
 '92 minutes',
 '69 minutes',
 '81 minutes',
 ['60 minutes (VHS and Wild Discovery version)', '71 minutes (original)'],
 '127 minutes',
 '93 minutes',
 '76 minutes',
 '75 minutes',
 '73 minutes',
 '85 minutes',
 '81 minutes',
 '70 minutes',
 '90 min.',
 '80 minutes',
 '75 minutes',
 '83 minutes',
 '83 minutes',
 '72 minutes',
 '97 minutes',
 '75 minutes',
 '104 minutes',
 '93 minutes',
 '105 minutes',
 '95 minutes',
 '97 minutes',
 '134 minutes',
 '69 minutes',
 '92 minutes',
 '126 minutes',
 '79 minutes',
 '97 minutes',
 '128 minutes',
 '73 minutes',
 '91 minutes',
 '105 minutes',
 '98 minutes',
 '130 minutes',
 '89 minutes',
 '93 minutes',
 '67 minutes',
 '98 minutes',
 '100 minutes',
 '118 m

In [124]:
def minute_to_integer(running_time):
  if running_time == 'N/A':
    return None

  if isinstance(running_time, list):
    entry = running_time[0]
    value = int(entry.split(" ")[0])
    return value
  else:
    value = int(running_time.split(" ")[0])
    return value



for movie in movie_info_list:
  movie["Running time (int)"] = minute_to_integer(movie.get('Running time', 'N/A'))


In [125]:
movie_info_list[0]

{'title': 'Snow White and the Seven Dwarfs',
 'Directed by': ['David Hand',
  'William Cottrell',
  'Wilfred Jackson',
  'Larry Morey',
  'Perce Pearce',
  'Ben Sharpsteen'],
 'Written by': ['Ted Sears',
  'Richard Creedon',
  'Otto Englander',
  'Dick Rickard',
  'Earl Hurd',
  'Merrill De Maris',
  'Dorothy Ann Blank',
  'Webb Smith'],
 'Based on': ['Snow White', 'by The', 'Brothers Grimm'],
 'Produced by': 'Walt Disney',
 'Starring': ['Adriana Caselotti',
  'Lucille La Verne',
  'Harry Stockwell',
  'Roy Atwell',
  'Pinto Colvig',
  'Otis Harlan',
  'Scotty Mattraw',
  'Billy Gilbert',
  'Eddie Collins',
  'Moroni Olsen',
  'Stuart Buchanan'],
 'Music by': ['Frank Churchill', 'Paul Smith', 'Leigh Harline'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': ['December 21, 1937 ( Carthay Circle Theatre )',
  'February 4, 1938 (United States)'],
 'Running time': '83 minutes',
 'Country': 'United States',
 'Language': 'English',

In [126]:
[movie.get('Running time (int)', 'N/A') for movie in movie_info_list]

[83,
 88,
 126,
 74,
 64,
 70,
 42,
 70,
 71,
 75,
 94,
 73,
 75,
 82,
 68,
 74,
 96,
 75,
 84,
 77,
 92,
 69,
 81,
 60,
 127,
 93,
 76,
 75,
 73,
 85,
 81,
 70,
 90,
 80,
 75,
 83,
 83,
 72,
 97,
 75,
 104,
 93,
 105,
 95,
 97,
 134,
 69,
 92,
 126,
 79,
 97,
 128,
 73,
 91,
 105,
 98,
 130,
 89,
 93,
 67,
 98,
 100,
 118,
 103,
 110,
 80,
 74,
 91,
 91,
 97,
 118,
 139,
 131,
 92,
 87,
 116,
 93,
 110,
 110,
 131,
 101,
 108,
 84,
 78,
 75,
 164,
 106,
 110,
 99,
 113,
 108,
 102,
 85,
 91,
 93,
 100,
 100,
 79,
 96,
 113,
 89,
 117,
 92,
 88,
 92,
 87,
 93,
 93,
 93,
 90,
 83,
 96,
 88,
 89,
 91,
 93,
 92,
 97,
 100,
 100,
 89,
 None,
 91,
 112,
 115,
 95,
 91,
 97,
 104,
 74,
 48,
 77,
 104,
 128,
 101,
 94,
 104,
 90,
 100,
 88,
 93,
 98,
 112,
 84,
 97,
 97,
 114,
 96,
 97,
 109,
 83,
 90,
 107,
 96,
 103,
 91,
 95,
 105,
 113,
 80,
 101,
 90,
 74,
 90,
 89,
 110,
 74,
 93,
 84,
 83,
 74,
 77,
 107,
 93,
 88,
 108,
 84,
 121,
 89,
 104,
 90,
 86,
 84,
 108,
 107,
 96,
 98,
 105,


#### Convert Budget & Box office to number


In [127]:
[movie.get('Budget', 'N/a') for movie in movie_info_list]

['$1.49 million',
 '$2.6 million',
 '$2.28 million',
 '$600,000',
 '$950,000',
 '$858,000',
 'N/a',
 '$788,000',
 'N/a',
 '$1.35 million',
 '$2.125 million',
 'N/a',
 '$1.5 million',
 '$1.5 million',
 'N/a',
 '$2.2 million',
 '$1,800,000',
 '$3 million',
 'N/a',
 '$4 million',
 '$2 million',
 '$300,000',
 '$1.8 million',
 'N/a',
 '$5 million',
 'N/a',
 '$4 million',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 '$700,000',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 '$6 million',
 'under $1 million or $1,250,000',
 'N/a',
 '$2 million',
 'N/a',
 'N/a',
 '$2.5 million',
 'N/a',
 'N/a',
 '$4 million',
 '$3.6 million',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 '$3 million',
 'N/a',
 '$3 million',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 '$3 million',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 '$4.4–6 million',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 '$4 million',
 'N/a',
 '$5 million',
 'N/a',
 'N/a',
 'N/a',
 'N/a',
 '$5 milli

TODO

* Given either a string ora list of strings as input, return a number (int or float) which is equal to the montary value

* money_conversion("$12.2 million") --> 12200000  ## Word Syntax

* money_coversaion("$790,000") --> 790000  ## Value Syntax

In [128]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

value_re = rf"\${number}"
word_re = rf"\${number}(-|\sto\s)?({number})?\s({amounts})"

# print(re.search(number, '704,222.12').group())

def word_to_value(word):
  value_dict = {"thousand" : 1000, "million":1000000, "billion": 1000000000}
  return value_dict[word]

def parse_value_syntax(string):
  value_string = re.search(number, string).group()
  value = float(value_string.replace(",",""))
  # strip out commas before returningsolution
  return value

def parse_word_syntax(string):
  value_string = re.search(number, string, flags=re.I).group()
  value = float(value_string.replace(",",""))
  word = re.search(amounts, string).group().lower()
  word_value = word_to_value(word)
  return value*word_value

def money_conversion(money):
  if money == 'N/A':
    return None
  if isinstance(money, list):
    money = money[0]
  value_syntax = re.search(value_re, money)
  word_syntax = re.search(word_re, money)
  if word_syntax:
    return parse_word_syntax(word_syntax.group())
  elif value_syntax:
    return parse_value_syntax(value_syntax.group())
  else:
    return None


In [129]:
for movie in movie_info_list:
  movie['Budget (float)'] = money_conversion(movie.get('Budget', 'N/A'))
  movie['Box office (float)'] = money_conversion(movie.get('Box office', 'N/A'))

In [130]:
movie_info_list[1]

{'title': 'Pinocchio',
 'Directed by': ['Ben Sharpsteen',
  'Hamilton Luske',
  'Bill Roberts',
  'Norman Ferguson',
  'Jack Kinney',
  'Wilfred Jackson',
  'T. Hee'],
 'Story by': ['Ted Sears',
  'Otto Englander',
  'Webb Smith',
  'William Cottrell',
  'Joseph Sabo',
  'Erdman Penner',
  'Aurelius Battaglia'],
 'Based on': ['The Adventures of Pinocchio', 'by', 'Carlo Collodi'],
 'Produced by': 'Walt Disney',
 'Starring': ['Cliff Edwards',
  'Dickie Jones',
  'Christian Rub',
  'Walter Catlett',
  'Charles Judels',
  'Evelyn Venable',
  'Frankie Darro'],
 'Music by': ['Leigh Harline', 'Paul J. Smith'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': ['February 7, 1940 ( Center Theatre )',
  'February 23, 1940 (United States)'],
 'Running time': '88 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.6 million',
 'Box office': '$164 million',
 'Running time (int)': 88,
 'Budget (float)': 2600000.0,
 

#### Convert Dates to DateTime object

In [131]:
[movie.get('Release date', 'N/A') for movie in movie_info_list]

['N/A',
 'N/A',
 ['November 13, 1940'],
 ['June 27, 1941'],
 'N/A',
 'N/A',
 'N/A',
 ['July 17, 1943'],
 'N/A',
 'N/A',
 'N/A',
 ['September 27, 1947'],
 'May 27, 1948',
 'N/A',
 ['October 5, 1949'],
 'N/A',
 'N/A',
 'N/A',
 'N/A',
 ['February 5, 1953 (United States)'],
 ['July 23, 1953 (US)'],
 ['November 10, 1953'],
 'N/A',
 ['August 17, 1954'],
 ['December 23, 1954'],
 'May 25, 1955',
 ['June 22, 1955'],
 ['September 14, 1955'],
 'December 22, 1955',
 'June 8, 1956',
 'July 18, 1956',
 ['September 4, 1956'],
 ['December 20, 1956'],
 'June 19, 1957',
 'August 28, 1957',
 ['December 25, 1957'],
 ['July 8, 1958'],
 ['August 12, 1958'],
 ['December 25, 1958'],
 ['January 29, 1959'],
 ['March 19, 1959'],
 'N/A',
 ['November 10, 1959'],
 'January 21, 1960 ( Sarasota, FL )',
 ['February 24, 1960'],
 'May 19, 1960',
 'N/A',
 ['November 1, 1960'],
 ['December 21, 1960'],
 ['January 25, 1961'],
 'March 16, 1961',
 ['June 21, 1961'],
 ['July 12, 1961'],
 ['July 17, 1961'],
 ['December 14, 1961

In [132]:
from datetime import datetime

dates = [movie.get('Release date', 'N/A') for movie in movie_info_list]

def clean_date(date):
  return date.split("(")[0].strip()

def date_conversion(date):
  if isinstance(date, list):
    date = date[0]
  if date == 'N/A':
    return None
  date_str = clean_date(date)
  fmts = ["%B %d, %Y", "%d %B %Y", "Y"]
  for fmt in fmts: 
    try:
      return datetime.strptime(date_str, fmt)
    except:
      pass
  return None

In [133]:
for movie in movie_info_list: 
  movie['Release date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

In [134]:
movie_info_list[2]

{'title': 'Fantasia',
 'Directed by': ['Samuel Armstrong',
  'James Algar',
  'Bill Roberts',
  'Paul Satterfield',
  'Ben Sharpsteen',
  'David D. Hand',
  'Hamilton Luske',
  'Jim Handley',
  'Ford Beebe',
  'T. Hee',
  'Norman Ferguson',
  'Wilfred Jackson'],
 'Story by': ['Joe Grant', 'Dick Huemer'],
 'Produced by': ['Walt Disney', 'Ben Sharpsteen'],
 'Starring': ['Leopold Stokowski', 'Deems Taylor'],
 'Narrated by': 'Deems Taylor',
 'Cinematography': 'James Wong Howe',
 'Music by': 'See program',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['November 13, 1940'],
 'Running time': '126 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.28 million',
 'Box office': '$76.4–$83.3 million (United States and Canada)',
 'Running time (int)': 126,
 'Budget (float)': 2280000.0,
 'Box office (float)': 83300000.0,
 'Release date (datetime)': datetime.datetime(1940, 11, 13, 0, 0)}

In [135]:
import pickle

def save_data_pickle(name, data):
  with open(name, 'wb') as f:
    pickle.dump(data, f)

In [136]:
import pickle

def load_data_pickle(name):
  with open(name, 'rb') as f:
    return pickle.load(f)

In [137]:
save_data_pickle('disney_movie_cleaned_data.pickle', movie_info_list)

### Task#4: Attach IMDB/Rotten Tommatoes/Metascore ratings Using API

In [138]:
import requests
import os
import urllib

# http://www.omdbapi.com/?apikey=[yourkey]&

def get_omdb_info(title):
  base_url = "http://www.omdbapi.com/?"
  parameters = {'apikey': '2704f42a', 't': title}
  params_encoded = urllib.parse.urlencode(parameters)
  full_url = base_url + params_encoded
  return requests.get(full_url).json()

def get_rotten_tomatoes_score(omdb_info):
  ratings = omdb_info.get('Ratings', [])
  for rating in ratings:
    if rating['Source'] == 'Rotten Tomatoes':
      return rating['Value']
  return None

get_omdb_info

<function __main__.get_omdb_info(title)>

In [139]:
for movie in movie_info_list:
  title = movie['title']
  omdb_info = get_omdb_info(title)
  movie['imdb'] = omdb_info.get('imdbRating', None)
  movie['metascore'] = omdb_info.get('Metascore', None)
  movie['rotten_tomatoes'] = get_rotten_tomatoes_score(omdb_info)

In [140]:
movie_info_list[2]

{'title': 'Fantasia',
 'Directed by': ['Samuel Armstrong',
  'James Algar',
  'Bill Roberts',
  'Paul Satterfield',
  'Ben Sharpsteen',
  'David D. Hand',
  'Hamilton Luske',
  'Jim Handley',
  'Ford Beebe',
  'T. Hee',
  'Norman Ferguson',
  'Wilfred Jackson'],
 'Story by': ['Joe Grant', 'Dick Huemer'],
 'Produced by': ['Walt Disney', 'Ben Sharpsteen'],
 'Starring': ['Leopold Stokowski', 'Deems Taylor'],
 'Narrated by': 'Deems Taylor',
 'Cinematography': 'James Wong Howe',
 'Music by': 'See program',
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release date': ['November 13, 1940'],
 'Running time': '126 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.28 million',
 'Box office': '$76.4–$83.3 million (United States and Canada)',
 'Running time (int)': 126,
 'Budget (float)': 2280000.0,
 'Box office (float)': 83300000.0,
 'Release date (datetime)': datetime.datetime(1940, 11, 13, 0, 0),
 'imdb': '7.7',
 'metasc

### Task#5: Save data in CSV/JSON format


In [141]:
# changing the datetime object to string to save the file
for movie in movie_info_list:
  current_date = movie['Release date (datetime)']
  if current_date:
    movie['Release date (datetime)'] = current_date.strftime('%B %d, %Y')
  else:
    movie['Release date (datetime)'] = None

In [142]:
import json

def save_data(name, data):
  with open(name, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

In [143]:
save_data('disney_data_final.json', movie_info_list)

In [144]:
import pandas as pd

df = pd.DataFrame(movie_info_list)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    520 non-null    object 
 1   Directed by              516 non-null    object 
 2   Written by               215 non-null    object 
 3   Based on                 283 non-null    object 
 4   Produced by              507 non-null    object 
 5   Starring                 481 non-null    object 
 6   Music by                 510 non-null    object 
 7   Production company       211 non-null    object 
 8   Distributed by           518 non-null    object 
 9   Release dates            185 non-null    object 
 10  Running time             502 non-null    object 
 11  Country                  458 non-null    object 
 12  Language                 496 non-null    object 
 13  Budget                   322 non-null    object 
 14  Box office               4

In [145]:
df.to_csv('disney_data_final.csv')